# Lab 3: Similarity Search & Distance Metrics**Estimated Time:** 6-7 minutes---

## Step 1: Your First Vector SearchRemember "bridge shaking"? Let's try it with vector search.

In [ ]:
print("=== VECTOR SEARCH: 'bridge shaking' (Cosine) ===\n")
run_query("""
    SELECT c.chunk_id,
           kb.doc_id,
           SUBSTR(kb.title, 1, 55) AS doc_title,
           ROUND(VECTOR_DISTANCE(
               c.embedding,
               VECTOR_EMBEDDING(doc_model USING 'bridge shaking'),
               COSINE
           ), 4) AS distance
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    ORDER BY distance
    FETCH EXACT FIRST 5 ROWS ONLY
""")

In [ ]:
print("=== VECTOR SEARCH: 'water contamination risk' ===\n")
run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 55) AS doc_title,
           kb.category,
           ROUND(VECTOR_DISTANCE(
               c.embedding,
               VECTOR_EMBEDDING(doc_model USING 'water contamination risk'),
               COSINE
           ), 4) AS distance
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    ORDER BY distance
    FETCH EXACT FIRST 5 ROWS ONLY
""")

## Step 2: Compare Distance MetricsOracle supports three distance metrics. Let's compare them on the same query.

In [ ]:
print("=== COSINE DISTANCE ===\n")
run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 45) AS doc_title,
           ROUND(VECTOR_DISTANCE(c.embedding,
               VECTOR_EMBEDDING(doc_model USING 'power outage impact on city infrastructure'),
               COSINE), 4) AS cosine_dist
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    ORDER BY cosine_dist
    FETCH EXACT FIRST 5 ROWS ONLY
""")

In [ ]:
print("=== DOT PRODUCT DISTANCE ===\n")
run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 45) AS doc_title,
           ROUND(VECTOR_DISTANCE(c.embedding,
               VECTOR_EMBEDDING(doc_model USING 'power outage impact on city infrastructure'),
               DOT), 4) AS dot_dist
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    ORDER BY dot_dist
    FETCH EXACT FIRST 5 ROWS ONLY
""")

In [ ]:
print("=== EUCLIDEAN DISTANCE ===\n")
run_query("""
    SELECT c.chunk_id,
           SUBSTR(kb.title, 1, 45) AS doc_title,
           ROUND(VECTOR_DISTANCE(c.embedding,
               VECTOR_EMBEDDING(doc_model USING 'power outage impact on city infrastructure'),
               EUCLIDEAN), 4) AS euclidean_dist
    FROM city_knowledge_chunks c
    JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
    ORDER BY euclidean_dist
    FETCH EXACT FIRST 5 ROWS ONLY
""")

In [ ]:
print("=== SIDE-BY-SIDE COMPARISON ===")
print("Query: 'power outage impact on city infrastructure'\n")

for metric in ['COSINE', 'DOT', 'EUCLIDEAN']:
    with connection.cursor() as cursor:
        cursor.execute(f"""
            SELECT c.chunk_id, SUBSTR(kb.title, 1, 40) AS title
            FROM city_knowledge_chunks c
            JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
            ORDER BY VECTOR_DISTANCE(c.embedding,
                VECTOR_EMBEDDING(doc_model USING 'power outage impact on city infrastructure'),
                {metric})
            FETCH EXACT FIRST 3 ROWS ONLY
        """)
        rows = cursor.fetchall()
    print(f"{metric:10s} → {', '.join(r[1].strip() for r in rows)}")

## Step 3: Approximate vs. Exact Search

In [ ]:
import time

query = 'emergency procedure for sensor malfunction'

# Exact search
start = time.time()
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT c.chunk_id, SUBSTR(kb.title, 1, 50) AS title
        FROM city_knowledge_chunks c
        JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
        ORDER BY VECTOR_DISTANCE(c.embedding,
            VECTOR_EMBEDDING(doc_model USING :query), COSINE)
        FETCH EXACT FIRST 5 ROWS ONLY
    """, {"query": query})
    exact_results = cursor.fetchall()
exact_time = time.time() - start

# Approximate search
start = time.time()
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT c.chunk_id, SUBSTR(kb.title, 1, 50) AS title
        FROM city_knowledge_chunks c
        JOIN city_knowledge_base kb ON c.doc_id = kb.doc_id
        ORDER BY VECTOR_DISTANCE(c.embedding,
            VECTOR_EMBEDDING(doc_model USING :query), COSINE)
        FETCH APPROXIMATE FIRST 5 ROWS ONLY
    """, {"query": query})
    approx_results = cursor.fetchall()
approx_time = time.time() - start

print(f"EXACT search:       {exact_time:.4f}s")
for r in exact_results:
    print(f"  chunk {r[0]}: {r[1]}")

print(f"\nAPPROXIMATE search: {approx_time:.4f}s")
for r in approx_results:
    print(f"  chunk {r[0]}: {r[1]}")

print(f"\nWith our small dataset, both are fast.")
print("With millions of vectors, approximate search + an index is essential.")

Your vector search is working. **Proceed to Lab 4** to add an HNSW index.